### ML

Используя наработки первой части (1) Анализ датасета Additional_services_Flights обучим наш датасет

#### Подготовка данных для обучения

In [26]:
import numpy as np         # библиотека для матриц и математики
import pandas as pd        # библиотека дл работы с табличками
from scipy import stats    # модуль для работы со статистикой

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
data = pd.read_excel("Problem1.xlsx", sheet_name = 'Обучение')
print(data.shape)

(1750, 10)


Напишем функцию для создания признаков для обучения

In [28]:
def create_features(data):
    
    # Запишем начальную дату полученную из исследования (1)
    min_date = pd.to_datetime('2016-10-01 00:00:00')
    
    # Добавим новый признак Месяц рейса
    data['Месяц рейса'] = data['Дата рейса'].apply(lambda x: x.month)
    
    # Бинарную переменную ВС переведем в 0 и 1
    data['ВС'].replace(['ВС1','ВС2'],[0,1],inplace=True)
    
    # Закодируем переменную Продолжительность рейса с помощью one-hot-encodind
    flight_duration=data['Продолжительность рейса']
    flight_duration= pd.get_dummies(flight_duration)
    # Удалим один столбец во избежания dummy-ловушки 
    flight_duration = flight_duration.drop(['от 3 до 6 часов'], axis=1)
    
    data = pd.concat((data, flight_duration), axis=1)
    
    # Закодируем переменную Тип услуги с помощью one-hot-encodind
    services=data['Тип услуги']
    services= pd.get_dummies(services)
    # Удалим один столбец во избежания dummy-ловушки 
    services = services.drop(['C'], axis=1)
    
    # В реальной тестовой выборке у нас не все месяца, поэтому чтобы
    # функция работала и на ней, будем кодировать Месяц рейса с учетом этой особенности
    months=data['Месяц рейса']
    min_months = data['Месяц рейса'].min()
    max_months = data['Месяц рейса'].max()
    
    # Закодируем месяца до мин месяца
    for index in range(1,min_months):
        index_name = str(index)
        data[index_name]=0
    
    # Закодируем месяца, которые есть в датасете
    months= pd.get_dummies(months)    
    data = pd.concat((data, months), axis=1)
    
    # Закодируем месяца после макс месяца
    for index in range(max_months+1,12):
        index_name = str(index)
        data[index_name]=0
    
    # Если в данных есть все 12 месяцев, то удалим один столбец во избежание dummy-ловушки
    if (min_months == 1) and (max_months == 12):
        data = data.drop([12], axis=1)
    
    data = pd.concat((data, services), axis=1)
    
    # Сделаем центрирование и нормирование столбца Цена
    data['Цена, руб std'] = (data['Цена, руб']-data['Цена, руб'].mean())/data['Цена, руб'].std()
    
    # Введем новый признак, количество дней с первой даты нашего датасета
    data['Day from date'] = data['Дата рейса'] - min_date   
    data['Day from date']=data['Day from date'].map(lambda x: str(x)[:-14])
    data['Day from date'] = data['Day from date'].astype(int)
    
    # Сделаем центрирование и нормирование столбца Day from date
    data['Day from date std'] = (data['Day from date']-data['Day from date'].mean())/data['Day from date'].std()
    
    
    X = data.drop(['Мест на рейсе','Цена, руб','Месяц рейса', 'Доступных мест услуги',
                             'Продолжительность рейса','Куда','Откуда','Дата рейса','Тип услуги','Day from date'], axis=1)
    return X

In [29]:
def create_target(data):
    data['Относительный спрос']=data['Фактический спрос']/data['Доступных мест услуги']
    return data['Относительный спрос']

In [30]:
X = create_features(data)
X = X.drop(['Фактический спрос'], axis=1)
y = create_target(data)

In [31]:
X

,ВС,< 3 часов,> 6 часов,1,2,3,4,5,6,7,8,9,10,11,A,F,S,SU,"Цена, руб std",Day from date std
0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-0.353690,1.591867
1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.542066,1.591867
2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.752168,1.591867
3,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-0.796033,1.591867
4,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-0.353690,1.582595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.349644,-3.461549
1746,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-0.396972,-3.470822
1747,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-0.396972,-3.470822
1748,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.349644,-3.470822


In [32]:
y

0       0.844444
1       0.851852
2       0.461538
3       0.444444
4       0.933333
          ...   
1745    0.148148
1746    0.000000
1747    0.000000
1748    0.000000
1749    0.042553
Name: Относительный спрос, Length: 1750, dtype: float64

#### Обучение с помощью Линейной регрессии

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression().fit(X_train, y_train)
mse = mean_squared_error(y_test,lin_reg.predict(X_test))
mse

0.032958830058687036

####  Обучение с L2 регуляризацией Ridge

In [40]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, cross_val_score

alpha_array = [0.01, 0.1, 1, 10, 100, 500, 1000]

grid_searcher = GridSearchCV(
    Ridge(),
    param_grid={'alpha':alpha_array}, 
    cv=5, n_jobs=-1)
grid_searcher.fit(X_train, y_train)
print('Best cross-validation parameters:',grid_searcher.best_params_)

Best cross-validation parameters: {'alpha': 0.01}


In [41]:
ridge = Ridge(alpha=0.01).fit(X_train, y_train)
mean_squared_error(y_test,ridge.predict(X_test))

0.033176097676314305

Видим, что L2 регуляризиция уменьшения MSE не показала, поэтому на реальной тестовой выборке будем использовать LinearRegression

#### Запись данных для реальной тестовой выборке

In [35]:
data_test = pd.read_excel("Problem1.xlsx", sheet_name = 'Тест')
print(data_test.shape)

(162, 9)


In [42]:
X_test_real = create_features(data_test)
X_test_real

,ВС,< 3 часов,> 6 часов,1,2,3,4,5,6,7,8,9,10,11,A,F,S,SU,"Цена, руб std",Day from date std
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-0.630584,-1.597996
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-0.154686,-1.597996
2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-0.916123,-1.597996
3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-0.757490,-1.491023
4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-0.471951,-1.491023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.081779,1.504230
158,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-0.889612,1.504230
159,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-0.827028,1.504230
160,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-0.920904,1.504230


In [43]:
y_test_real_predict = lin_reg.predict(X_test_real)

In [48]:
y_test_real_predict

array([0.60395563, 0.65072459, 0.72588047, 1.04549377, 1.0714135 ,
       0.93948908, 0.86392042, 0.83512071, 0.91068938, 1.04549377,
       0.73035387, 0.30948342, 1.07638005, 1.10537701, 0.89857433,
       0.94563918, 1.04712855, 0.86957737, 0.73504426, 0.73951766,
       1.05922959, 1.08532685, 0.95458599, 0.87852418, 0.84952721,
       0.92558902, 1.05922959, 0.74399106, 0.32300226, 1.08980026,
       1.11879722, 0.91199454, 0.95905939, 1.06054876, 0.88299758,
       0.62671716, 0.67378201, 0.74846446, 1.09427366, 0.96353279,
       0.88747098, 0.75293787, 0.75741127, 1.0771232 , 1.10322046,
       0.9724796 , 0.89641779, 0.86742082, 0.94348264, 1.0771232 ,
       0.76188467, 0.34089587, 1.10769387, 1.13669083, 0.92988815,
       0.976953  , 1.07844237, 0.90089119, 0.76635808, 0.77083148,
       1.09054341, 1.11664067, 0.90983799, 0.88084103, 0.95690284,
       1.09054341, 0.77530488, 0.35431608, 1.12111408, 1.15011104,
       0.94330836, 0.99037321, 1.09186258, 0.9143114 , 0.77977

Видим, что часть значений имеют предсказание Относительного спроса >1, что в нашей задачи не имеет физического смысла (нельзя купить больше услуг, чем количество Доступных мест услуги.  
Поэтому ограничим такую возможность

In [46]:
def limit_for_precision(data_precision):
    for index in range(0,len(data_precision)):
        if data_precision[index] > 1:
            data_precision[index] = 1
    return data_precision

In [49]:
y_test_real_predict_limit = limit_for_precision(y_test_real_predict)

Теперь запишем наши предсказания в Тестовый датасет и округлим значения до целых

In [53]:
data_test['Предсказание - Фактический спрос'] = y_test_real_predict_limit*data_test['Доступных мест услуги']
data_test['Предсказание - Фактический спрос'] = data_test['Предсказание - Фактический спрос'].astype(int)

In [54]:
data_test

,Дата рейса,Откуда,Куда,Продолжительность рейса,ВС,Мест на рейсе,Тип услуги,Доступных мест услуги,"Цена, руб",Месяц рейса,Предсказание - Фактический спрос
0,2018-04-01,VKO,LCA,от 3 до 6 часов,0,522,S,39,1460.0,4,23
1,2018-04-01,VKO,LCA,от 3 до 6 часов,0,522,SU,27,2555.0,4,17
2,2018-04-01,VKO,PRG,< 3 часов,1,189,F,9,803.0,4,6
3,2018-04-02,VKO,BKK,> 6 часов,0,522,A,45,1168.0,4,45
4,2018-04-02,VKO,HKT,> 6 часов,0,522,A,45,1825.0,4,45
...,...,...,...,...,...,...,...,...,...,...,...
157,2018-04-30,VKO,BKK,> 6 часов,0,522,SU,27,5400.0,4,27
158,2018-04-30,LED,LCA,от 3 до 6 часов,0,522,A,45,864.0,4,45
159,2018-04-30,VKO,BKK,> 6 часов,0,522,A,45,1008.0,4,45
160,2018-04-30,VKO,PRG,< 3 часов,1,189,F,9,792.0,4,7


In [55]:
data_test.to_csv('Предсказание_Фактический спрос.csv') 